In [7]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc read-aloud"><h2>--- Day 10: Balance Bots ---</h2><p>You come upon a factory in which many robots are zooming around handing small microchips to each other.</p>

<p>Upon closer examination, you notice that each bot only proceeds when it has <em>two</em> microchips, and once it does, it gives each one to a different bot or puts it in a marked "output" bin. Sometimes, bots take microchips from "input" bins, too.</p>
<p>Inspecting one of the microchips, it seems like they each contain a single number; the bots must use some logic to decide what to do with each chip. You access the local control computer and download the bots' instructions (your puzzle input).</p>
<p>Some of the instructions specify that a specific-valued microchip should be given to a specific bot; the rest of the instructions indicate what a given bot should do with its <em>lower-value</em> or <em>higher-value</em> chip.</p>
<p>For example, consider the following instructions:</p>
<pre><code>value 5 goes to bot 2
bot 2 gives low to bot 1 and high to bot 0
value 3 goes to bot 1
bot 1 gives low to output 1 and high to bot 0
bot 0 gives low to output 2 and high to output 0
value 2 goes to bot 2
</code></pre>
<ul>
<li>Initially, bot <code>1</code> starts with a value-<code>3</code> chip, and bot <code>2</code> starts with a value-<code>2</code> chip and a value-<code>5</code> chip.</li>
<li>Because bot <code>2</code> has two microchips, it gives its lower one (<code>2</code>) to bot <code>1</code> and its higher one (<code>5</code>) to bot <code>0</code>.</li>
<li>Then, bot <code>1</code> has two microchips; it puts the value-<code>2</code> chip in output <code>1</code> and gives the value-<code>3</code> chip to bot <code>0</code>.</li>
<li>Finally, bot <code>0</code> has two microchips; it puts the <code>3</code> in output <code>2</code> and the <code>5</code> in output <code>0</code>.</li>
</ul>
<p>In the end, output bin <code>0</code> contains a value-<code>5</code> microchip, output bin <code>1</code> contains a value-<code>2</code> microchip, and output bin <code>2</code> contains a value-<code>3</code> microchip. In this configuration, bot number <em><code>2</code></em> is responsible for comparing value-<code>5</code> microchips with value-<code>2</code> microchips.</p>
<p>Based on your instructions, <em>what is the number of the bot</em> that is responsible for comparing value-<code>61</code> microchips with value-<code>17</code> microchips?</p>
</article>


In [10]:
instructions = """
value 5 goes to bot 2
bot 2 gives low to bot 1 and high to bot 0
value 3 goes to bot 1
bot 1 gives low to output 1 and high to bot 0
bot 0 gives low to output 2 and high to output 0
value 2 goes to bot 2
"""


def process(instructions: str, target: list[int]) -> dict[str, str]:
    bots = {}
    outputs = {}
    the_bot = None
    for line in instructions.strip().splitlines():
        if line.startswith("bot"):
            bot_fr, *bots_to = re.findall(r"\w+ \d+", line)
            if bot_fr not in bots:
                bots[bot_fr] = {"values": []}
            bots[bot_fr]["to"] = bots_to
            outputs |= {bot: [] for bot in bots_to if bot.startswith("output")}
        else:
            value, bot = re.findall(r"\w+ \d+", line)
            if bot not in bots:
                bots[bot] = {}
            if "values" not in bots[bot]:
                bots[bot]["values"] = []
            bots[bot]["values"].append(int(value.split()[1]))
            bots[bot]["values"].sort()

    change = True
    while change:
        change = False
        for bot in list(bots.keys()):
            if len(bots[bot]["values"]) == 2:
                if bots[bot]["values"] == target:
                    the_bot = bot
                change = True
                v_low, v_high = bots[bot]["values"][:2]
                bots[bot]["values"] = bots[bot]["values"][2:]
                bot_low, bot_high = bots[bot]["to"]
                if bot_low.startswith("bot"):
                    bots[bot_low]["values"].append(v_low)
                    bots[bot_low]["values"] = (
                        sorted(bots[bot_low]["values"][:2])
                        + bots[bot_low]["values"][2:]
                    )
                else:
                    outputs[bot_low].append(v_low)

                if bot_high.startswith("bot"):
                    bots[bot_high]["values"].append(v_high)
                    bots[bot_high]["values"] = (
                        sorted(bots[bot_high]["values"][:2])
                        + bots[bot_high]["values"][2:]
                    )
                else:
                    outputs[bot_high].append(v_high)

    return bots, outputs, the_bot


bots, outputs, the_bot = process(instructions, [2, 5])
# Initially,
#       bot 1 starts with a value-3 chip, and
#       bot 2 starts with a value-2 chip and a value-5 chip.
# Because bot 2 has two microchips:
#       it gives its lower one (2) to bot 1 and
#       its higher one (5) to bot 0.
# Then, bot 1 has two microchips;
#       it puts the value-2 chip in output 1 and
#       gives the value-3 chip to bot 0.
# Finally, bot 0 has two microchips;
#       it puts the 3 in output 2 and
#       the 5 in output 0.
print(the_bot)

bot 2


In [14]:
with open("../input//day10.txt") as f:
    instructions_f = f.read()

print("Part I: ", process(instructions_f, [17, 61])[2])

Part I:  bot 27


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>27</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p><span title="What do you get if you multiply six by nine?">What do you get</span> if you <em>multiply together the values</em> of one chip in each of outputs <code>0</code>, <code>1</code>, and <code>2</code>?</p>
</article>

</main>


In [27]:
from math import prod


_, outputs, _ = process(instructions_f, [17, 61])

prod(prod(vs) for k, vs in outputs.items() if re.search(r" [0-2]$", k))

13727

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>13727</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2016">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="10/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Balance+Bots%22+%2D+Day+10+%2D+Advent+of+Code+2016&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F10&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Balance+Bots%22+%2D+Day+10+%2D+Advent+of+Code+2016+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F10';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
